In [1]:
 ! pip install -q kaggle

In [ ]:
 ! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
!kaggle datasets download -d salader/dogs-vs-cats

 98% 1.05G/1.06G [00:13<00:00, 55.9MB/s]
100% 1.06G/1.06G [00:14<00:00, 81.4MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [ ]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [ ]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
# create CNN model

model = Sequential()

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(16,kernel_size=(3,3),padding='valid',activation='relu'))

model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 128)     3584      
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 128)    512       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      73792     
                                                                 
 batch_normalization_1 (Batc  (None, 125, 125, 64)     256       
 hNormalization)                                                 
                                                        

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
625/625 [==============================] - 157s 225ms/step - loss: 0.7368 - accuracy: 0.5689 - val_loss: 0.6816 - val_accuracy: 0.5302
Epoch 2/10
625/625 [==============================] - 139s 222ms/step - loss: 0.6640 - accuracy: 0.5734 - val_loss: 0.6821 - val_accuracy: 0.5286
Epoch 3/10
625/625 [==============================] - 139s 222ms/step - loss: 0.6657 - accuracy: 0.5774 - val_loss: 0.6702 - val_accuracy: 0.5478
Epoch 4/10
625/625 [==============================] - 139s 222ms/step - loss: 0.6613 - accuracy: 0.5781 - val_loss: 0.6551 - val_accuracy: 0.6192
Epoch 5/10
625/625 [==============================] - 139s 222ms/step - loss: 0.6493 - accuracy: 0.6056 - val_loss: 0.6530 - val_accuracy: 0.6042
Epoch 6/10
625/625 [==============================] - 139s 222ms/step - loss: 0.6242 - accuracy: 0.6492 - val_loss: 0.9560 - val_accuracy: 0.5928
Epoch 7/10
625/625 [==============================] - 138s 221ms/step - loss: 0.6016 - accuracy: 0.6806 - val_loss: 0.8208 -